In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go

app = dash.Dash(__name__)

# Load your dataset
df = pd.read_csv('F:/Docs/Data Science/Projects/final_dataset.csv')

# Handle missing values
df.fillna(0, inplace=True)

# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])
# Extract the year from the 'Date' column and create a 'Season' column
df['Season'] = df['Date'].dt.year
# Encode categorical variables (if needed)
df['FTR'] = df['FTR'].map({'H': 1, 'NH': 0})

# Calculate additional metrics
df['GoalDifference'] = df['FTHG'] - df['FTAG']
df['PointsDifference'] = df['HTP'] - df['ATP']

# Create a list of team options for the dropdown
team_options = [{'label': team, 'value': team} for team in df['HomeTeam'].unique()]

app.layout = html.Div([
    html.H1('Football Data Analysis Dashboard'),

    # Dropdown to select a specific team
    dcc.Dropdown(
        id='team-dropdown',
        options=team_options,
        value=team_options[0]['value']
    ),

    # Visualization 1: Bar chart of PL Winner each season
    dcc.Graph(id='bar-chart',
              config={'displayModeBar': False}),  # Disable the display mode bar for this chart

    # Visualization 2: Points Over Time for the selected team
    dcc.Graph(id='line-chart'),

    # Visualization 3: Goal Distribution for the selected team
    dcc.Graph(id='histogram'),

    # Visualization 4: Correlation between Goal Difference and Points Difference
    dcc.Graph(id='scatter-plot'),

    # Visualization 5: Heatmap for Home Team Form Points Over Matchweeks
    dcc.Graph(id='heatmap')
])

@app.callback(
    Output('bar-chart', 'figure'),
    Output('line-chart', 'figure'),
    Output('histogram', 'figure'),
    Output('scatter-plot', 'figure'),
    Output('heatmap', 'figure'),
    [Input('team-dropdown', 'value')]
)
def update_figures(selected_team):
    # Update figures here using your data and selected team

    # Replace the provided team name with the selected team
    team_data = df[df['HomeTeam'] == selected_team]

    # Visualization 1: Bar chart of PL Winner each season
    highest_points_teams = df.groupby(['Season', 'HomeTeam'])['HTP'].sum().reset_index()
    highest_points_teams = highest_points_teams.sort_values(by=['Season', 'HTP'], ascending=[True, False])
    highest_points_teams = highest_points_teams.groupby('Season').first().reset_index()
    fig1 = go.Figure(data=go.Bar(
        x=highest_points_teams['Season'].astype(str) + ' - ' + highest_points_teams['HomeTeam'],
        y=highest_points_teams['HTP']
    ))
    fig1.update_layout(
        xaxis_title='Season - Team',
        yaxis_title='Total Points',
        title='PL Winner each season (Highest Total Points per Season)',
        xaxis_tickangle=-45
    )

    # Visualization 2: Points Over Time for the selected team
    fig2 = go.Figure(data=go.Scatter(x=team_data['Date'], y=team_data['HTP'], mode='lines+markers'))
    fig2.update_layout(
        xaxis_title='Date',
        yaxis_title='Points',
        title=f'Points Over Time for {selected_team}'
    )

    # Visualization 3: Goal Distribution for the selected team
    fig3 = go.Figure()
    fig3.add_trace(go.Histogram(x=team_data['FTHG'], name='Home Goals', opacity=0.75))
    fig3.add_trace(go.Histogram(x=team_data['FTAG'], name='Away Goals', opacity=0.75))

    fig3.update_layout(
        xaxis_title='Goals',
        yaxis_title='Frequency',
        title=f'Goal Distribution for {selected_team}',
        barmode='overlay'
    )

    # Visualization 4: Correlation between Goal Difference and Points Difference
    fig4 = go.Figure(data=go.Scatter(x=df['GoalDifference'], y=df['PointsDifference'], mode='markers'))
    fig4.update_layout(
        xaxis_title='Goal Difference',
        yaxis_title='Points Difference',
        title='Correlation between Goal Difference and Points Difference'
    )

    # Visualization 5: Heatmap for Home Team Form Points Over Matchweeks
    form_matrix = df.pivot_table(index='HomeTeam', columns='MW', values='HTFormPts', aggfunc='first')
    fig5 = go.Figure(data=go.Heatmap(z=form_matrix.values, colorscale='Viridis'))
    fig5.update_layout(
        title='Home Team Form Points Over Matchweeks'
    )

    return fig1, fig2, fig3, fig4, fig5

if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\soumi\AppData\Local\Temp\ipykernel_18084\97605390.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\soumi\AppData\Local\Temp\ipykernel_18084\97605390.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\soumi\AppData\Local\Temp\ipykernel_18084\97605390.py:17: UserWarning:

Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.



OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.